In [ ]:
!pip install cellpose[gui] --quiet
!pip install opencv-python-headless moviepy


In [ ]:
from google.colab import files
uploaded = files.upload()

import cv2
import os
import numpy as np

uploaded_file = next(iter(uploaded))
video_path = uploaded_file
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frames.append(gray)
cap.release()
print(f"Total frames loaded: {len(frames)}")


In [ ]:
from cellpose import models, io
import matplotlib.pyplot as plt

model = models.Cellpose(gpu=False, model_type='cyto')
masks_all, flows, styles, diams = model.eval(frames, diameter=None, channels=[0,0])
print("Segmentation complete.")


In [ ]:
import pandas as pd
from skimage.measure import regionprops, label

def classify_frame(index, total):
    if index < total // 3:
        return 'Seed'
    elif index < 2 * total // 3:
        return 'Fixed'
    else:
        return 'Dead'

results = []
overlay_frames = []
mask_frames = []
colors = {
    "Seed": (0, 102, 255),
    "Fixed": (0, 255, 0),
    "Dead": (255, 0, 0),
    "Debris": (255, 255, 0)
}

for idx, (frame, mask) in enumerate(zip(frames, masks_all)):
    labeled = label(mask)
    props = regionprops(labeled, intensity_image=frame)
    cell_class = classify_frame(idx, len(frames))

    count = 0
    total_area = 0
    total_brightness = 0
    mask_color = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)

    for p in props:
        area = p.area
        brightness = p.mean_intensity
        if area < 10:
            debris_color = colors["Debris"]
            coords = p.coords
            for y, x in coords:
                mask_color[y, x] = debris_color
            results.append((idx, 'Debris', area, brightness))
        else:
            coords = p.coords
            color = colors[cell_class]
            for y, x in coords:
                mask_color[y, x] = color
            total_area += area
            total_brightness += brightness
            count += 1

    overlay = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    combined = cv2.addWeighted(overlay, 0.6, mask_color, 0.4, 0)

    overlay_frames.append(combined)
    mask_frames.append(mask_color)

    if count > 0:
        results.append((idx, cell_class, total_area/count, total_brightness/count, count))
    else:
        results.append((idx, cell_class, 0, 0, 0))

df = pd.DataFrame(results, columns=["Frame", "Class", "Avg Area", "Avg Brightness", "Count"])
df.to_csv("frame_statistics.csv", index=False)


In [ ]:
from moviepy.editor import ImageSequenceClip

overlay_clip = ImageSequenceClip([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in overlay_frames], fps=fps)
mask_clip = ImageSequenceClip([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in mask_frames], fps=fps)

overlay_clip.write_videofile("combined_overlay_report.mp4", codec='libx264', bitrate="5000k")
mask_clip.write_videofile("masks_only_report.mp4", codec='libx264', bitrate="5000k")


In [ ]:
from google.colab import files
files.download("frame_statistics.csv")
files.download("combined_overlay_report.mp4")
files.download("masks_only_report.mp4")
